<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 5: Pipeline and Grid Search

## Predicting Grant Applications: Introduction

### Lesson Objectives

* After completing this lesson, you should be able to:
  - Understand how to fit together the functions available in Spark's machine learning libraries to solve real problems
  - Use a Spark cluster to fit models in a fraction of the time
  
### Examples in thie Module

* This module contains 5 lessons
* Examples in later lessons follow on from previous lessons in this module
  - e.g. Examples in lesson 3 follow on from lessons 1 and 2
  
### Lesson Objectives

* After completing this lesson, you should be able to:
  - Understand, in a general way, the data available in the grant application dataset
  - Appreciate the problem being solved
  - Propose some features that might be good predictions
  
### The Dataset and Problem

* Dataset
  -8708 grant applications from the University of Melbourne, 3992 of which were successful
  - the 17594 researchers involved in those applications
  - <https://www.kaggle.com/c/unimelb>
* Problem
  - Predict if a grant application will be successful
  
### Data About Grants

* Grant_Application_ID (int)
* Grant_Status (int)
  - Whether or not the grant application was successful
* Contract_Value_Band (string)
  - The amount of the grant, bucketized into ranges
* Grant_Category_Code (string)
  - Categorization of the sponsor (e.g. Australian competitive grants, cooperative research centre, industry)
  
### Data Aobut Researchers

* Role (string)
  - The individual researcher's role in the propsed research
* Year_of_Birth (int)
* Dept_No (string)
* With_PHD (string)
* Number_of_Successful_Grant (int)
* Number_of_Unsuccessful_Grant (int)
* A2, A, B, C (four int columns)
  - Number of articles published in journals of various quality (A2 being the most prestigious, C the least)
  
### Basic Setup

In [1]:
import sys.process._
"wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0105EN/data/grantsPeople.csv  -P data/"!

--2019-12-19 14:03:11--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0105EN/data/grantsPeople.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3126338 (3.0M) [text/csv]
Saving to: ‘data/grantsPeople.csv’

     0K .......... .......... .......... .......... ..........  1%  198K 15s
    50K .......... .......... .......... .......... ..........  3% 1.08M 9s
   100K .......... .......... .......... .......... ..........  4%  467K 8s
   150K .......... .......... .......... .......... ..........  6% 1.03M 6s
   200K .......... .......... .......... .......... ..........  8% 1.05M 6s
   250K .......... .......... .......... .......... ..........  9% 1.05M 5s
   300K .......... .......... .......... 

import sys.process._

res0_1: Int = 0

In [2]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import $ivy.`org.apache.spark::spark-mllib:2.4.0` // Or use any other 2.x version here
import  org.apache.spark.SparkContext
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
val sc= new SparkContext("local[*]","Linear Methods")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                     // Or use any other 2.x version here

import  org.apache.spark.SparkContext

import org.apache.log4j.{Level, Logger}

sc: SparkContext = org.apache.spark.SparkContext@583ccaad

In [3]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._
import org.apache.spark.sql.functions._

import org.apache.spark.sql.SparkSession

spark: SparkSession = org.apache.spark.sql.SparkSession@24aece89
import spark.implicits._

import org.apache.spark.sql.functions._

### Loading the Data

In [5]:
val data = spark.read.
  format("com.databricks.spark.csv").
  option("delimiter", "\t").
  option("header", "true").
  option("inferSchema", "true").
  load("data/grantsPeople.csv")

data: org.apache.spark.sql.package.DataFrame = [Grant_Application_ID: int, RFCD_Code: string ... 22 more fields]

In [6]:
data.show()

+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+----+----+------------+------------+--------------------+-------------------+
|Grant_Application_ID| RFCD_Code|RFCD_Percentage| SEO_Code|SEO_Percentage|Person_ID|                Role|Year_of_Birth|Country_of_Birth|Home_Language| Dept_No|Faculty_No|With_PHD|No_of_Years_in_Uni_at_Time_of_Grant|Number_of_Successful_Grant|Number_of_Unsuccessful_Grant|  A2|   A|   B|   C|Grant_Status|Sponsor_Code| Contract_Value_Band|Grant_Category_Code|
+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+-

### Plan of Attack?

* Before continuing to the next lesson, think about
  - We want to have features that are good predictors for each grant
  - How can we create features about grants from the data about the researchers?
  - Which values in the dataset might make good predictors?
  

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.